# Picture book

Feed story lines and use generative AI to make the corresponding illustrations

## Import

In [ ]:
from IPython.display import display
from ipywidgets import interactive

from picturebook_app import DemoApp

## Interactive Demo

In [ ]:
system_template = """You are an AI conversing with a human. The human input is a line of a story. 
Respond with a short description of a cartoon illustration that could be used to illustrate the story so far.

Begin!
----------------"""

human_template = "{question}"

image_template = "children's cartoon: "

app = DemoApp(system_template=system_template, human_template=human_template, image_template=image_template)

def process_caption(caption=''):
    if app.sp.captions and caption == app.sp.captions[-1]:
        return
    app.sp.update(caption)
    with app.out:
        display(app.sp.cap_images[-1])
    

process_caption_button = interactive(process_caption, {"manual": True, "manual_name": "Process Caption"})
display(process_caption_button, app.out)

In [ ]:
app.save()

## Run a whole story

In [ ]:
one_fish_two_fish = """
One fish, Two fish, Red fish, Blue fish,
Black fish, Blue fish, Old fish, New fish.
---
This one has a little car.
This one has a little star.
Say! What a lot of fish there are.
---
Yes. Some are red, and some are blue.
Some are old and some are new.
---
Some are sad, and some are glad,
And some are very, very bad.
Why are they sad and glad and bad?
I do not know, go ask your dad.
---
Some are thin, and some are fat.
The fat one has a yellow hat.
---
From there to here,
From here to there,
Funny things are everywhere.
---
Here are some who like to run.
They run for fun in the hot, hot sun.
Oh me! Oh my! Oh me! oh my!
What a lot of funny things go by.
---
Some have two feet and some have four.
Some have six feet and some have more.
Where do they come from? I can't say.
But I bet they have come a long, long way.
---
we see them come, we see them go.
Some are fast. Some are slow.
Some are high. Some are low.
---
Not one of them is like another.
Don't ask us why, go ask your mother.
"""

robots = """
On the ground and in the air. Robots, robots, everywhere!
---
Up in space, beneath the seas, robots make discoveries!
"""

brown_bear = """
Brown bear, brown bear, what do you see?
I see a red bird looking at me.
---
Red bird, red bird, what do you see?
I see a yellow duck looking at me.
---
Yellow duck, yellow duck, what do you see?
I see a blue horse looking at me.
---
Blue horse, blue horse, what do you see?
I see a green frog looking at me.
---
Green frog, green frog, what do you see?
I see a purple cat looking at me.
"""

In [ ]:
system_template = """You are an AI conversing with a human. The human input is a line of a story. 
Respond with a short description of a cartoon illustration that could be used as an image generation prompt for stable diffusion to illustrate the story so far.

Begin!
----------------"""

human_template = "{question}"

image_template = "children's cartoon: "

app = DemoApp(system_template=system_template, human_template=human_template, image_template=image_template)
for verse in brown_bear.split("---"):
    app.sp.update(verse.strip())
    display(app.sp.cap_images[-1])
app.save()